In [1]:
import os
import glob
from pathlib import Path
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
plt.ion()   # interactive mode
from torch.utils.data import Dataset, DataLoader

In [152]:
import numpy as np
def gen_random_data(feature_name, num_rows, num_cols, categiries = None):
    mat = np.random.rand(num_rows,num_cols)
    mat = np.around(mat, decimals=3)
    np.savetxt('data/numeric/{}.csv'.format(feature_name), mat, delimiter=',')
gen_random_data('n4', 1000, 1300)

In [174]:
class InputDataset(Dataset):
    def __init__(self, data_dir = 'data'):
        """
        Args: data_dir (string): data dir containing numeric and categorical csvs.
        """
        # check data_dir exists
        if not (Path(data_dir)).exists():
            raise FileNotFoundError('directory not found: "{}"'.format(data_dir))

        # this is the main dataframe which will hold all the features
        self.df = pd.DataFrame()
        
        # read numeric csvs
        numeric_data_dir = data_dir + '/numeric'
        if not (Path(numeric_data_dir)).exists():
            raise FileNotFoundError("expected 'numeric' direcroty not found under: '{}'".format(data_dir))
        for f in glob.glob(numeric_data_dir + '/*.csv'):
            df = pd.read_csv(f, header=None)
            self.create_numeric_feature(f, df)
        
        # read categorical csvs
        categorical_data_dir = data_dir + '/categorical'
        if not (Path(categorical_data_dir)).exists():
            raise FileNotFoundError("expected 'categorical' direcroty not found under: '{}'".format(data_dir))
        for f in glob.glob(categorical_data_dir + '/*.csv'):
            df = pd.read_csv(f, header=None)
            self.create_categorical_feature(f, df)
    
    def __len__(self):
        return len(self.df.index)
    
    def __getitem__(self, idx):
        return self.df.iloc[idx]

    def create_numeric_feature(self, f, df):
        '''
        concatinate rows to form a single column vector
        '''
        # feature name == filename (without ext)
        feature_name = os.path.splitext(os.path.basename(f))[0]
        
        values = []
        df_tmp = pd.DataFrame()
        for i in range(len(df.index)):
            values.extend(df.iloc[i].values)
        df_tmp[feature_name] = pd.Series(values)
        self.df = pd.concat([self.df, df_tmp], axis=1)
        
    def create_categorical_feature(self, f, df):
        # feature name
        name = os.path.splitext(os.path.basename(f))[0]
        
ds = InputDataset()
print(ds.df.shape)
ds.df.head()

(1300000, 4)


,n3,n1,n4,n2
0,7.0,4.0,0.696,4.0
1,1.0,3.0,0.243,1.0
2,3.0,4.0,0.620,2.0
3,4.0,5.0,0.307,3.0
4,5.0,6.0,0.888,4.0


In [167]:
# df = pd.read_csv('data/numeric/n4.csv', header=None)
# print(df.shape)
# df.head()